## ДЗ 1 еще не сделано

In [7]:
from IPython import display
import numpy as np
import random
import torch
import math 


In [2]:
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

1.	Нахождение сложной производной.
Найдите производную по x от функции:
 
При этом надо пользоваться встроенным в PyTorch autograd.

В итоге я хочу видеть функцию вида:

def find_x_derivative(x, y):
    YOUR CODE HERE
  
   return # YOUR CODE HERE


In [18]:
x = torch.autograd.Variable(torch.Tensor([2]),requires_grad=True)
y = 5*x**4 + 3*x**3 + 7*x**2 + 9*x - 5

y.backward()
x.grad

tensor([233.])

In [17]:
x =1
y= 1

z = math.sin(math.tan(x)*x**2*y**(-1)+np.log(np.exp(-x*x-3)+x**3*y))*math.tan(x**2*np.exp(x**9))
f = torch.autograd.Variable(torch.Tensor([z]),requires_grad=True)
z 
f

tensor([-0.4505], requires_grad=True)

In [ ]:
def find_x_derivative(x, y):
   # YOUR CODE HERE
  
   return # YOUR CODE HERE

tensor([233.])